In [1]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-rec7cjud
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-rec7cjud
  Resolved https://github.com/huggingface/transformers.git to commit 3b8e2932ce743008f63585aae1e1b8b30dc8b3ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 45.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 27.8 MB/s eta 0:00:00
  

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline,AutoTokenizer,AutoModelForCausalLM
from datasets import load_dataset
from IPython.display import Audio
import transformers

2024-03-31 05:22:20.778947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 05:22:20.779074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 05:22:20.910498: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TypeError: expected string or bytes-like object

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Trial English --- dataset works only on CPU

In [ ]:
#dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
#sample = dataset[0]["audio"]
#result = pipe(sample)
#print(result["text"])


In [ ]:
model2 = "tiiuae/falcon-7b-instruct"
tokenizer2 = AutoTokenizer.from_pretrained(model2)
pipeline = transformers.pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenizer2,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [ ]:
sequences = pipeline(
    result["text"],
    max_length=1000,
    do_sample=True,
    top_k=1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

trial 2 Arabic 

In [32]:
#result = pipe('/kaggle/input/audio-arabic/V1.wav')
#print(result["text"])

 طيب معانا اتصال اخر من استاز احمد من الشرقية استاز احمد نواطي تلفزيون سأل نور اهلا بحضرتك بس نواطي صوت تلفزيون عشان اسمح حضرتك كوي نواطي يا فنان خلص مساء الخير يا دكتور اسامة مساء الخير استاز والله بس انا اسف دكتور اسامة اني بتكلم في وقت حضرتك على موضوع مادة الجلالة فعلا. اه اهلا اهلا بالفضل. واخشي ما احرق برضو دكتور اسامة في الموضوع بالنسبة جيف ليهان الشمال بهم النوم الصبح اه بحس انه مش قادر افتح معي. هم. تمامام تقدر وكنت استعملت مرهم التراميسين اللي هو العادي دوت وعملني التهابات تحت الجسد وإحبوط وحتى الان يعني مرقتش علو علو هو تقريبا الخط قطع


In [33]:
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [36]:
sequences = pipeline(
    result["text"],
    max_length=1000,
    do_sample=True,
    top_k=1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [37]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result:  طيب معانا اتصال اخر من استاز احمد من الشرقية استاز احمد نواطي تلفزيون سأل نور اهلا بحضرتك بس نواطي صوت تلفزيون عشان اسمح حضرتك كوي نواطي يا فنان خلص مساء الخير يا دكتور اسامة مساء الخير استاز والله بس انا اسف دكتور اسامة اني بتكلم في وقت حضرتك على موضوع مادة الجلالة فعلا. اه اهلا اهلا بالفضل. واخشي ما احرق برضو دكتور اسامة في الموضوع بالنسبة جيف ليهان الشمال بهم النوم الصبح اه بحس انه مش قادر افتح معي. هم. تمامام تقدر وكنت استعملت مرهم التراميسين اللي هو العادي دوت وعملني التهابات تحت الجسد وإحبوط وحتى الان يعني مرقتش علو علو هو تقريبا الخط قطع علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقريبا علو هو تقري�ا علو هو تق�ل علو ه�لا استعملت مرهم ال�راميسين ال�ادي دوت وع�لت مرهم ال�راميسين ال�ادي د�لت مرهم ال�راميسين ال��امي د�لت مرهم ال�راميسي� د�لت مرهم ال�رام� د�لت مرهم ال�ر�اب د�ل� مرهم ال�ر�اب د�ل� مرهم ال��امي د�ل� مرهم